<a href="https://colab.research.google.com/github/DonRoboto/KalmanFilter_v1/blob/main/SKF_v2_Covar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bitstring

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import random
import bitstring as bits
from sympy import *
import scipy, pylab

In [3]:
x = Symbol('x')
y = Symbol('y')

#f1 = (4*x)**2 + (4*y)**2
f1 = (x-5)**2 + (y-5)**2

In [4]:
class Individuo():
    
    #inicializar con valores aleatorios
    def __init__(self, i, x='', y=''):
      self.i = i
      self.z_x = 0
      self.z_y = 0

      self.x_gorro=0
      self.y_gorro=0

      if x!='':
        self.f_x = float(x)        
        self.f_y = float(y)        
      else:
        self.f_x = random.uniform(-500, 500)        
        self.f_y = random.uniform(-500, 500)
                
      #convertir las coordenadas de posicion a binario
      self.cromosomas()

    #coordenadas como coromosomas
    def cromosomas(self):
        aux_f_x = bits.BitArray(float=self.f_x, length=32)
        aux_f_y = bits.BitArray(float=self.f_y, length=32)
        
        self.b_x = list(aux_f_x.bin)
        self.b_y = list(aux_f_y.bin)
         
    #funcion de aptitud
    def fitness(self):
      func_1 = f1
      #func_2 = f2

      self.cromosomas()

      r1_1 = ''.join(self.b_x)
      r1_2 = ''.join(self.b_y)
      
      r2_1 = bits.BitArray(bin=r1_1).float
      r2_2 = bits.BitArray(bin=r1_2).float
      
      result_1 = func_1.subs([(x, r2_1), (y, r2_2)]).doit()
      #result_2 = func_2.subs([(x, r2_1), (y, r2_2)]).doit()

      #return result_1, result_2
      return result_1
      

    #posicion actual
    def posicion(self):
      x = self.f_x
      y = self.f_y
           
      return x, y

In [5]:
class Poblacion():
  p = []

  def __init__(self, n=''):
    self.p = []
    if n!='':
      for k in range(n):
        i=Individuo(i=k)
        self.p.append(i)

In [6]:
pos_test = []
seq_test = []
n_pob = 30
#random.seed(4)
pob1 = Poblacion(n_pob)

for i in range(n_pob):
  #fit1, fit2 = pob1.p[i].fitness()
  x1, y1 = pob1.p[i].posicion()   
  fit1 = pob1.p[i].fitness()
  #seq_test.append([round(fit1, 2), round(fit2, 2)])   
  pos_test.append([round(x1, 2), round(y1, 2)])   
  seq_test.append([round(fit1, 2)])   
  #print(fit1)
  #print(fit2)
  #print("\n")

print(pos_test)
print(seq_test)



[[-404.8, 296.73], [-372.35, 422.16], [119.15, 271.72], [-478.75, 100.92], [315.39, -181.92], [-17.81, 231.23], [169.21, 63.48], [458.78, -376.89], [-71.39, -404.94], [-363.85, -329.58], [-264.71, 60.58], [122.64, -491.59], [-330.85, -153.83], [274.7, 56.59], [-432.59, -192.43], [387.07, 343.21], [111.09, -135.61], [324.81, 406.73], [409.97, 263.02], [12.3, -334.01], [445.57, -130.86], [497.67, -245.78], [241.39, 464.72], [-137.4, -142.64], [375.44, 261.16], [416.6, 297.61], [309.09, -228.88], [305.46, 403.93], [-446.19, -445.38], [-193.39, -266.72]]
[[253039.06], [316410.55], [84168.67], [243218.90], [131281.98], [51702.64], [30386.65], [351750.02], [173887.04], [247994.91], [75833.21], [260440.18], [138021.37], [75398.09], [230464.73], [260360.67], [31027.03], [263663.45], [230569.41], [114983.56], [212557.45], [305620.65], [267222.53], [42075.81], [202848.06], [255038.19], [147168.97], [249420.91], [406420.29], [113192.45]]


In [7]:
#prediction and prediction noise covariance
Px = 1000
Py = 1000
Q = 0.5
R = 0.5

#los best (minimizar)
f_true = 1000000
x_true = 1000000
y_true = 1000000


n_time = 50
for t in range(n_time):

  for i in range(n_pob):    
    if pob1.p[i].fitness()<f_true:
      f_true = pob1.p[i].fitness()
      x_true = pob1.p[i].f_x
      y_true = pob1.p[i].f_y
      
  for i in range(n_pob):
    #predict
    pob1.p[i].x_gorro = pob1.p[i].f_x    
    pob1.p[i].y_gorro = pob1.p[i].f_y    
    Px = Px + Q
    Py = Py + Q

    #measure
    rnd_x = random.uniform(0, 1)
    rnd_y = random.uniform(0, 1)
    pob1.p[i].z_x = pob1.p[i].x_gorro + np.sin(rnd_x * 2 * np.pi) * np.abs(pob1.p[i].x_gorro-x_true)
    pob1.p[i].z_y = pob1.p[i].y_gorro + np.sin(rnd_y * 2 * np.pi) * np.abs(pob1.p[i].y_gorro-y_true)
    
    #estimate and update the agents’ position
    Kx = Px / (Px + R)
    Ky = Py / (Py + R)
    pob1.p[i].f_x = pob1.p[i].x_gorro + Kx * (pob1.p[i].z_x - pob1.p[i].x_gorro)
    pob1.p[i].f_y = pob1.p[i].y_gorro + Ky * (pob1.p[i].z_y - pob1.p[i].y_gorro)


  #calcular el error covariance matrix
  errx = []
  erry = []
  for e in range(n_pob):
    errx.append(pob1.p[i].f_x-x_true)
    erry.append(pob1.p[i].f_y-y_true)

  Px = np.cov(errx)
  Py = np.cov(erry)

  Px = (1-Kx)*Px
  Py = (1-Ky)*Py

  #print(x_true)
  print(f_true)


30386.6506111082
9354.39664978982
4224.97434844294
4224.97434844294
1008.70104693525
286.414770971922
286.414770971922
127.147215654004
7.55333903372434
7.55333903372434
7.55333903372434
7.55333903372434
7.55333903372434
7.55333903372434
6.18330237760443
6.18330237760443
6.18330237760443
6.18330237760443
0.672726755178019
0.672726755178019
0.579370071460517
0.0405324275948260
0.0405324275948260
0.0405324275948260
0.0405324275948260
0.0118222177266034
0.00151415145887768
0.00151415145887768
0.00151415145887768
0.000642110704802690
0.000642110704802690
0.000642110704802690
0.000404151095835914
0.000116232120490167
6.46099078949192e-5
1.33897033265384e-5
8.25954998617817e-6
1.78385039362183e-6
1.78385039362183e-6
1.78385039362183e-6
1.46503521136765e-6
1.00745751296927e-6
1.00745751296927e-6
1.00745751296927e-6
2.33777427638415e-7
2.33777427638415e-7
2.33777427638415e-7
2.33756964007625e-7
1.67420921570738e-7
1.51032963913167e-8
